# Adjusting Audio Channel Gains on Programmable Logic

This notebook will make use of a custom IP created in System Generator. The IP applies adjustable gains to the two audio channels on the Pynq-Z2 programmable logic, the values of which can be controlled from this notebook via interactive Widgets.

The purpose of the notebook is to provide an introduction into developing hardware designs for use with Pynq as well as offer some insight into the audio components specific to the Pynq-Z2 development board. Although gains are used in this example, the model is made general enough that these can be replaced by any digital signal processing technique desired. 

 
## Audio Format and Hardware Connections

The Pynq-Z2 incorporates an Analog Devices ADAU1761 audio codec. The codec sits between the two audio jacks on the board (LINE IN and HP + MIC) and the FPGA Package Pins on the Zynq XC7Z020. Figure 1 illustrates the hardware connections.

![](./assets/HardwareConnections.jpg)
<center>Figure 1: Zynq SoC and Audio Codec Connectivity</center><br>

The precision which the codec accepts and outputs is shown in Figure 2. <br>

![](./assets/NumericPrecisionAndClippingStructure.jpg)
<center>Figure 2: Numeric Precision and Clipping Structure for Analog Devices ADAU1761 [1]</center><br>

The notation __1.23__ at the input and output of the codec means that the overall wordlength is 24 bits with 1 sign bit and 23 fractional bits. This gives a range of 0.999999880790710 down to -1 using twos complement.

The codec communicates with the Zynq via 9 pins. Table 1 provides information on how the codec and Zynq are connected. Naming conventions relative to the codec are on the left hand side of the table and the Zynq SoC on the right hand side. The middle column details the data that is transferred. The table has been created using information from the avaialable Pynq-Z2 schematics as well as information found in the ADAU1761 data sheet [2][1]. <br>

<center>Table 1: ADAU1761/ZynqConnections</center>

![](./assets/CodecZynqConnectionTable.jpg) <br>

Two rows are highlighted in red. These pins communicate using the I2C standard and are wired directly to the processing system via EMIO. They can be used to configure various aspects of the codec and have been included for information but will not be explored further in this notebook. 

Audio data is sent from the codec (26) into the FPGA (F17) and output from the FPGA (G18) to the codec (27). Only 1 pin is required to pass audio into or out of the FPGA. This is because the audio data is in a __serial format__. Figure 3 illustrates how this works.<br>

![](./assets/I2CMode-24bitsperchannel.jpg)
<center>Figure 3: Serial Data (I2C Mode) - 24 bits per channel [1]</center><br>

Each 24 bit word of data is sent as a consecutive stream of single bits. There are actually 32 bits sent on the hardware but only 24 of these contain audio information, the least significant bits are truncated internally by the codec. The reason for this padding is to provide an audio sampling rate of ~48 kHz. The default mode is I2C mode where the most significant bit (MSB) falls on the second BCLK pulse as shown in Figure 3. BCLK is a bit clock and is used to clock the serial input and output ports such that each clock cycle indicates a different bit in the word. LRCLK is the frame clock and clocks the serial input and output ports such that a high signal indicates the right channel and a low signal the left. 

## System Generator Model 

The System Generator model has five input ports and three output ports. The widths are specified in parentheses. 

#### Inputs: 
> SDATA In (1) <br>
> BCLK In (1) <br>
> LRCLK In (1) <br>
> Left Gain In (4) <br>
> Right Gain In (4)
    
#### Outputs:
> SDATA Out (1) <br>
> BCLK Out (1) <br>
> LRCLK Out (1)

By including the two clock signals, BCLK and LRCLK, each can be delayed by the same amount as the audio data. Thereby ensuring all signals are correctly aligned at the output and properly interpreted by the codec as in Figure 3.

The model is split into four distinct sections: 
1. Serial to Parallel & Channel Demultiplexing
2. Reinterpretation of Audio Channels 
3. Conventional DSP (in this case adjustable gains)
4. Channel Multiplexing & Parallel to Serial

### Serial to Parallel and Channel Seperation

In order to perform conventional digital signal processing on the audio data, the serial format must be parallelised. That is, the 24 bits of audio data must be represented using a data bus as oppossed to a single line. In addition, the two channels must be spereated into two distinct buses to be processed individually. Figure 4 shows the annotated model designed to acheive this. <br>

![](./assets/STPandChannelSplit.jpg)
<center>Figure 4: Converting Serial Audio Data into Parallel Form and Demultiplexing Channels</center><br>

#### Down-Sample
The system is clocked at 100 MHz which is 32 times faster than BCLK. In order for the serial to parallel block to correctly convert, it must read each of the bits relative to the BCLK rate. Otherwise it would interpret the serial stream wrong (there would be 32 'copies' of each bit in the sequence). Therefore, the SDATA line is down-sampled by 32. This rate change is also applied to LRCLK to make the delays easier to track. A further down-sample of 32 then occurs when the serial data is parallelised giving an overall down-sample of 1024. BCLK can only be down-sampled by 16 in order to obey Nyquist and delays along this line should therefore be 64 times greater.

#### Delay 
The serial to parallel block continusously converts 32 consecutive bits into a 32 bit wordlength bus as soon as the system is clocked. As there is no option to inform the block when a word begins, it must be ensured that the serial data enters the block at the correct time such that the output is the intended word. To acheive this, ideally, the single bit stream would enter this block as soon as the system was clocked. However, in practice this cannot be acheived as the down-sampling and input registers both have a combined latency of 33 causing an offset. To rectify this, an additional delay of 991 is included to bring the total delay to 1024. This causes the single bit stream to now align correctly with the serial to parallel conversion as this is the total number of clock cycles (at 100 MHz) per word. 

#### Left and Right Channels
The LRCLK is used as a control signal to enable the two serial to parallel blocks. This ensures that the top line always carries the left channel and the line below the right. 

### Reinterpretation of Audio Channels
Before performing any processing on the two audio channels they must be interpreted correctly. Currently they are interpreted by other blocks as unsigned 32-bit integers. Figure 5 shows how this is acheived. <br>

![](./assets/reinterpret.jpg)
<center>Figure 5: Reinterpret Audio Channels into Signed Fixed Point</center><br>

#### Slice 
Referring back to Figure 3, the first bit of the 32 is redundant and does not hold any information. The slice block removes this bit and outputs a 31-bit unsigned integer. 

#### Reinterpret
Figure 2 shows that the data output by the codec is signed (1 sign bit and 23 fractional). The reinterpret block reinterprets the output of the slice as a signed fixed point number with 1 sign bit and 31 fractional. The additional 8 fractional bits can be removed if wanting to save on resources. They will always be zero at this point. 

### Conventional DSP (Adjustable Gains)
The adjustable gains are simply two multipler blocks - one for each channel - illustrated in Figure 6. An important note is that the output of this section should have a wordlength of 32 in order to correctly align with the clock signals when serialised again. <br>

![](./assets/ConvDSP.jpg)
<center>Figure 6: Adjustable Gains Implemented as Multiplies</center><br>

#### Gain Input
The two channels are multipled by values input into the IP by two Gateway Ins. These gateways are sampled at the same rate as the multiplers. The precision of the gateways is UFix_4_3 with the intention that the gains will be limited to the range of 0 to 1 in this notebook. This ensures that the output does not exceed the codec's range and saturate. 

#### AXI4-Lite
For values to easily be passed into these ports from this notebook, the interface on each Gateway In is configured to be AXI4-Lite. Doing this makes the ports addressable and allows existing functions to be used from within the notebook to write values to each port. 

The address offsets are explicitly set within each block:

> Left Gain In Address Offset: 0x00 <br>
> Right Gain In Address Offset: 0x04

### Channel Multiplexing & Parallel to Serial
Finally, the two channels must be combined onto one line and serialised again to be output to the codec. Figure 7 illustrates how this is acheived in System Generator. <br>

![](./assets/ChannelMuxandSerial.jpg)
<center>Figure 7: Reinterpret Audio Channels into Signed Fixed Point</center><br>

#### Multiplexer
To combine both channels onto a signal line, the Mux block is used. LRCLK is again used as a control signal to switch between the two channels at the correct time. 

#### Parallel to Serial
Serialising the data does not require the same attention to latency as parellising as the codec is being informed by BCLK and LRCLK where words start and finish. 

#### Resampling
All signals are finally up-sampled to the system rate again before being output. 

## Vivado Block Design

The IP is integrated into the vivado block design as shown in Figure 8.<br>

![](./assets/VivadoBD.jpg)
<center>Figure 8: Block Design Integrating Audio Processing System Generator IP</center><br>

The serial audio data is input to the IP from pin F17 on the Zynq. The BCLK and LRCLK signals are taken from the output of the _audio codec controller_ Pynq IP, delayed by an appropriate amount, and finally output to audio codec (pins R18, T17 respectively). The serial audio data is both output to the codec via pin G18 for playback on the hardware as well as into the _audio codec controller_ IP to make use of existing infrastructure and be read into the processing system. 

## Import Overlay

In [ ]:
import pynq.lib.audio
from pynq import Overlay
audioHW = Overlay('./bitstream/audio_gain.bit')

## Setup 
Running the cell immediately below will bring up some information about the Overlay _audioHW_. Under the heading 'IP Blocks', two IPs should be visable. These correspond directly to the Vivado block diagram. audio_codec_ctrl_0 is a Pynq IP developed by Xilinx to control the codec. The other is the custom IP created for this tutorial.

In [ ]:
?audioHW

The cell below instantiates the two IPs found on the programmable logic as Python objects. This enables  the functionality of each IP to be accessed from within the Python environment in this notebook. 

In [ ]:
pAudio = audioHW.audio_codec_ctrl
pAudio.configure()

gains = audioHW.audio_gain

Select `LINE_IN` as the input port for audio. The other option is `HP+MIC`, however this tutorial will be using that port as the output. 

In [ ]:
pAudio.select_line_in()

## Functions
Two functions are created below. 
<br>The __setGains__ function makes use of _DefaultIP.write_ to write to memory mapped I/O; where defaultIP is an "IP without a more specific driver" (in this case our custom IP instantiated in Python as _gains_). The first argument is the address offset of the input and the second argument is the value. The value is expected to be an integer and will be provided by slider widgets here as the variables Lch and Rch. Because the inputs are in the form UFix_4_3, 8 corresponds to a value of 1 so when multiplied with the slider values will convert these into the correct format. 

In [ ]:
def setGains(Lch, Rch):
    try:
        gains.write(0x0, int(Lch*8))  # left Channel
        gains.write(0x4, int(Rch*8))  # Right Channel
    except Exception:
        import traceback
        print(traceback.format_exc())

The __sliders__ function uses _widgets.interact_ to create user interface controls for function arguments. When the controls are manipulated, the function associated with the controls is called and the values on the controls are passed into the corresponding function as arguments[3]. Here, the __setGains__ function above is called and the gains for each channel come from the user interface controls.

In [ ]:
def sliders():
    try:
        widgets.interact(setGains, Lch=LCH_Slider, Rch=RCH_Slider)
    except Exception:
        import traceback
        print(traceback.format_exc())

## Widgets
The following code details how to set up slider Widgets. The step size of 0.125 corresponds to the 3 fractional bits assigned to the two Gateway In blocks (_Left Gain In, Right Gain In_) within the System Generator model. A maximum gain of 1 ensures that that the audio data output does not exceed the codec's range and saturate. 

In [ ]:
import ipywidgets as widgets

LCH_Slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.125,
    description='Left Channel',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

RCH_Slider = widgets.FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.125,
    description='Right Channel',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)

## Output Audio and Adjust Gains

To be able to adjust the gains while the _pAudio.bypass()_ function is running, __multithreading__ has to be implemented. This allows for both functions to be run simultaneously. Otherwise the notebook would wait for one function to run before executing the other. 

In [ ]:
import _thread

outputLength = 60    # Specify how long the audio should be output in seconds (0 to 60)

try:
    _thread.start_new_thread(pAudio.bypass, (outputLength,))
    _thread.start_new_thread(sliders, ())
except Exception:
    import traceback
    print(traceback.format_exc())

## References

[1] https://www.analog.com/media/en/technical-documentation/data-sheets/ADAU1761.pdf

[2] http://www.tul.com.tw/download/TUL_PYNQ%20Schematic_R12.pdf

[3] https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html